In [1]:
import fastai
import random
import torch

from fastai.tabular import *

In [2]:
%load_ext autoreload
%autoreload 1
%aimport logs

In [3]:
# Check if we're using the GPU.
torch.cuda.get_device_name(0)

'GeForce GTX 1080'

This next section is training the model. You don't have to do this if the model is already trained.

In [5]:
fname = logs.character_filename("ironclad")
df = pd.read_csv(fname)
df.head()

,Character,Floor,Deck Size,Choice1,Choice2,Choice3,Picked,A Thousand Cuts,A Thousand Cuts+1,AbeCurse,...,Vajra,Velvet Choker,VioletLotus,War Paint,WarpedTongs,Whetstone,White Beast Statue,WingedGreaves,WristBlade,Yang
0,IRONCLAD,1,11,Headbutt,Iron Wave,Entrench,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,IRONCLAD,2,12,Ghostly Armor,Body Slam,Dual Wield,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,IRONCLAD,4,14,Pommel Strike,Perfected Strike,Burning Pact,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,IRONCLAD,5,15,Wild Strike,Second Wind,Fire Breathing,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,IRONCLAD,6,16,Warcry,Flex,Flame Barrier,3,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [6]:
size = len(df)
procs = [Categorify, Normalize]
dep_var = "Picked"
cat_names = ["Character", "Choice1", "Choice2", "Choice3"]
valid_idx = sorted(random.sample(list(range(size)), int(size / 5)))

In [7]:
data = TabularDataBunch.from_df(fname, df, dep_var, valid_idx=valid_idx, procs=procs, cat_names=cat_names)

In [8]:
learn = tabular_learner(data, layers=[200,100], metrics=accuracy)

In [9]:
learn.fit_one_cycle(1, 0.01)

epoch     train_loss  valid_loss  accuracy  time    
0         0.998445    115610.2734380.576721  00:08     


In [12]:
learn.fit_one_cycle(1, 0.001)

epoch     train_loss  valid_loss  accuracy  time    
0         0.943560    64.770348   0.585118  00:07     


In [13]:
learn.export(logs.cachefile("ironclad.learn"))

This next section is testing out the model on sample data. The `load_learner` call is loading a cached learner instead of training a new one.

In [4]:
learn = load_learner(logs.CACHE, "ironclad.learn")

In [7]:
deck = ["Strike_R"] * 3 + ["Defend_R"] * 4 + [
    "Bash+1", "Anger", "AscendersBane", "Disarm+1", "Headbutt", "Evolve", "Impervious", "Whirlwind+1", "Shockwave",
    "Demon Form+1", "Heavy Blade+1", "Dark Embrace+1", "Headbutt+1", "Sever Soul+1", "Clothesline+1", "Corruption"]
relics = ["Burning Blood", "Cursed Key"]
floor = 36
choices = ["Perfected Strike+1", "Sword Boomerang+1", "Armaments"]
testcsv = logs.mini_csv("IRONCLAD", floor, deck, relics, choices)
testf = pd.read_csv(testcsv)
p = learn.predict(testf.iloc[0])
p

(Category tensor(1), tensor(1), tensor([0.0170, 0.6341, 0.0650, 0.2839]))

In [16]:
list(p[2].numpy())

[0.0169597, 0.634105, 0.064994425, 0.28394082]